In [ ]:
from simu_PSF_polar import *
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from matplotlib.colors import LogNorm

from tqdm import tqdm

In [ ]:
def gauss(coords, A, mu1, mu2, s, c):
    x, y = coords
    return A*(1/(2*np.pi*(s**2)))*np.exp(-0.5*(((x-mu1)**2+(y-mu2)**2)/((s**2)))) + c

In [ ]:
def fit_psf_in_pixel(data_raw):
    uuuu, vvvv = np.meshgrid(np.arange(data_raw.shape[0]),np.arange((data_raw.shape[0])))
    xdata = np.vstack((uuuu.ravel(), vvvv.ravel()))
    zdata = data_raw.ravel()
    p, pcov = curve_fit(gauss, xdata, zdata, p0=(200, 20, 20, 2, 0))
    return p, gauss((uuuu, vvvv), p[0], p[1], p[2], p[3], p[4])

In [ ]:
na=1.4
nnn = 200
rcut = na/n1
rr = np.linspace(-rcut, rcut, nnn)
xx, yy = np.meshgrid(rr, rr)

k = 2*np.pi/0.617
m, n = np.meshgrid(range(-nnn+1, nnn), range(-nnn+1, nnn))
dk = na/nnn
th = np.arcsin((dk/n1)*np.sqrt(m**2+n**2))
th[np.sin(th)>na/n1]=0.
th[np.isnan(th)]=0.
phi = np.arctan2(n,m)
Kx = m*dk
Ky = n*dk

In [ ]:
th = np.arcsin(np.sqrt(xx**2+yy**2))
th[np.sqrt(xx**2+yy**2)>rcut]=np.nan

In [ ]:
plt.rcParams['figure.figsize'] = [7, 5]
meeeesh = plt.pcolormesh(th)
cb = plt.colorbar(meeeesh, pad=0.15, label='Photon number')

In [ ]:
f_tube = 200000
k = 2*np.pi/0.617
f_o = f_tube/100
Dx = 2*rcut*f_o/nnn
l_pixel=4
Npadding = int((2*np.pi*f_tube)/(k*n1*l_pixel*Dx)) - nnn 
if Npadding%2==1:
    Npadding=Npadding+1
lambd = 617

In [ ]:
#Npadding = (0.001*lambd*f_tube-l_pixel*2*rcut*f_o)/(Dx*l_pixel)

In [ ]:
print(Npadding)

In [ ]:
bfp0 = 1/(np.cos(th))**(1/2)
bfp0[np.isnan(bfp0)] = 0.

In [ ]:
bfp = np.zeros((nnn+Npadding, nnn+Npadding), dtype='float64')
bfp[Npadding//2:Npadding//2+nnn,Npadding//2:Npadding//2+nnn] = bfp0

In [ ]:
plt.rcParams['figure.figsize'] = [7, 5]
meeeesh = plt.pcolormesh(bfp)
cb = plt.colorbar(meeeesh, pad=0.15, label='Photon number')

In [ ]:
field = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(bfp)))

In [ ]:
psf = np.real(field)**2 + np.imag(field)**2

In [ ]:
plt.rcParams['figure.figsize'] = [15, 5]
fig, ax = plt.subplots(1,2)
NN = len(psf)
to_fit = psf[NN//2-21:NN//2+22,NN//2-21:NN//2+22]/np.sum(psf[NN//2-21:NN//2+22,NN//2-21:NN//2+22])
p, sim = fit_psf_in_pixel(to_fit)
vmax2 = np.max(sim)
vmax = np.max(to_fit)
print(vmax)
aa, bb = np.meshgrid(np.arange(to_fit.shape[0])*l_pixel, np.arange(to_fit.shape[0])*l_pixel)
me = ax[0].pcolormesh(aa, bb, to_fit, norm=LogNorm(vmin=1e-6, vmax=0.028))
cb = plt.colorbar(me, pad=0.15, label='Photon number')
ax[0].plot(l_pixel*(21+((1*0.61)*lambd/(10*l_pixel*na))*np.cos(np.linspace(0, 2*np.pi, 100))), l_pixel*(21+((1*0.61)*lambd/(10*l_pixel*na))*np.sin(np.linspace(0, 2*np.pi, 100))), c='r')
mesh2 = ax[1].imshow(sim, vmin=0, vmax=vmax2)
cb = plt.colorbar(mesh2, pad=0.15, label='Photon number')
ax[0].set_xlabel('x_camera ($\mu$m)')
ax[0].set_ylabel('y_camera ($\mu$m)')

In [ ]:
plt.rcParams['figure.figsize'] = [5, 2]
med = (to_fit[22]+ to_fit[23])/2
plt.plot(med)
plt.grid()
print((31-11)*10*l_pixel, 2.44*(lambd/na))

In [ ]:
mesh2 = plt.imshow(sim-to_fit)
cb = plt.colorbar(mesh2, pad=0.15, label='Photon number')